## PŘÍPRAVA DAT (PREPROCESSING)

In [13]:
import pandas as pd
from unidecode import unidecode
from sklearn.preprocessing import StandardScaler
import unicodedata
import re

# Funkce pro odstranění diakritiky
def remove_diacritics(text):
    return unidecode(text)

# Funkce pro úpravu názvů sloupců
def fix_column_names(df):
    df.columns = [unidecode(col).replace(' ', '_').lower() for col in df.columns]
    return df

# Načtení datového souboru
file_path = 'NAPO_VSE_oprava.xlsx'
df = pd.read_excel(file_path, sheet_name='Labels')

# Úprava názvů sloupců
df = fix_column_names(df)

# Převedení numerických hodnot na int a float
df = df.convert_dtypes()

# Funkce pro normalizaci textu v buňkách
def normalize_text(text):
    if pd.isna(text):
        return text
    # Odstranění diakritiky, převod na malá písmena, nahrazení whitespace za "_"
    text = ''.join(c for c in unicodedata.normalize('NFD', str(text)) if unicodedata.category(c) != 'Mn')
    text = text.replace(' ', '_').lower()
    return text

# Aplikace normalizace textu na všechny buňky v DataFrame
df = df.applymap(normalize_text)

# Zobrazení prvních několika řádků pro ověření
print(df.head())

# Úložení předzpracovaných dat do nového excelu
df.to_excel('NAPO_VSE_preprocessed.xlsx', index=False)

print("Předzpracovaná data byla uložena do 'NAPO_VSE_preprocessed.xlsx'")

C:\Users\Hoang\AppData\Local\Temp\ipykernel_7384\3932291866.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_text)


  id_respondenta pohlavi vek vekova_kategorie  \
0    025y2-0ce-1    zena  39        35-44_let   
1    032y2-0ce-1    zena  42        35-44_let   
2    04gy2-0ce-1     muz  60        55-64_let   
3    05l03-0ce-1    zena  56        55-64_let   
4    06vx2-0ce-1     muz  73    65_a_vice_let   

                   nejvyssi_dokoncene_vzdelani  vzdelani_\n(rekod)  \
0      vyssi_odborne,_vysokoskolske_bakalarske              vs/vos   
1  stredoskolske_s_maturitou_(vcetne_nastavby)      ss_s_maturitou   
2            vysokoskolske_magisterske_a_vyssi              vs/vos   
3                       vyucen(a)_bez_maturity  zs/ss_bez_maturity   
4  stredoskolske_s_maturitou_(vcetne_nastavby)      ss_s_maturitou   

               kraj jakou_znamkou_byste_ohodnotil(a)_ceske_zdravotnictvi?  \
0  stredocesky_kraj                                                  2      
1  stredocesky_kraj                                                  3      
2      ustecky_kraj                                  

### Kategorizace otázky Jaký je čistý měsíční příjem Vaší domácnosti?

In [14]:
# Funkce pro kategorizaci příjmu
def categorize_income(income):
    if income in ['do_19_999_kc']:
        return 'nizky_prijem'
    elif income in ['20_000_kc_-_49_999_kc', '50_000_kc_-_74_999_kc']:
        return 'stredni_prijem'
    elif income in ['75_000_kc_-_99_999_kc', '100_000_kc_-_149_000_kc', '150_000_kc_a_vice']:
        return 'vysoky_prijem'
    elif income in ['odmitam_odpovedet', 'nevim']:
        return 'neurceno'
    else:
        return 'n'

# Aplikace funkce na sloupec a vytvoření nové kategorie
df['prijem_kategorie'] = df['jaky_je_cisty_mesicni_prijem_vasi_domacnosti?'].apply(categorize_income)

# Zobrazení prvních několika řádků pro ověření
print(df[['jaky_je_cisty_mesicni_prijem_vasi_domacnosti?', 'prijem_kategorie']].head())


  jaky_je_cisty_mesicni_prijem_vasi_domacnosti? prijem_kategorie
0                         50_000_kc_-_74_999_kc   stredni_prijem
1                         20_000_kc_-_49_999_kc   stredni_prijem
2                             odmitam_odpovedet         neurceno
3                                  do_19_999_kc     nizky_prijem
4                         50_000_kc_-_74_999_kc   stredni_prijem


### Oprava odpovědí k otázce Jakou známkou byste ohodnotil/a české zdravotnictví?

In [15]:
# Funkce pro převod textových hodnot hodnocení na integer podle první číslice
def convert_rating_to_int(value):
    if pd.isna(value):
        return value
    match = re.match(r'(\d)_', value)
    if match:
        return int(match.group(1))
    else:
        try:
            return int(value)
        except ValueError:
            return pd.NA

# Seznam sloupců pro konverzi
columns_to_convert = [
    'jakou_znamkou_byste_ohodnotil(a)_ceske_zdravotnictvi?',
    'hodnoceni_zdravotni_pojistovny',
    'hodnoceni_zdravotnicka_zarizeni_/_lekari',
    'hodnoceni_ministerstvo_zdravotnictvi',
    'hodnoceni_nemocnice', 
    'hodnoceni_prakticky_lekar',
    'hodnoceni_ambulantni_specialiste'
]

# Aplikace konverze na vybrané sloupce
for col in columns_to_convert:
    df[col] = df[col].apply(convert_rating_to_int)

# Převedení numerických hodnot na int a float
df = df.convert_dtypes()

# Uložení upravených dat do nového Excel souboru
df.to_excel('NAPO_VSE_preprocessed.xlsx', index=False)

print("Data s opravou hodnocení byla uložena do 'NAPO_VSE_preprocessed.xlsx'")

Data s opravou hodnocení byla uložena do 'NAPO_VSE_preprocessed.xlsx'
